In [ ]:
"""Тестирование проводим за период с 18.12.2012 по 29.04.2022, 
т.к. с сайта Мосбиржи доступна подробная статистика по ребалансировкам для этого периода.
Можно сделать с 04.03.2011, но нет инф-ции по капитализации для тестирования фактора Size,
поэтому будут только Momentum, Low volatility и High dividends"""

***Создание свода о ребалансировках наших индексов + цены с дивидендами***

In [ ]:
# Добавить комиссии за ребалансировку!

In [559]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math

In [560]:
rebalancing = pd.read_excel('moex - assets.xlsx', sheet_name='динамика для бэктеста', header=1)
price = pd.read_excel('moex - assets.xlsx', sheet_name='цены', index_col=0).iloc[1:]
price = price.fillna(0)
dividends = pd.read_excel('moex - assets.xlsx', sheet_name='дивиденды').iloc[:, 1:]
Nshares = pd.read_excel('moex - assets.xlsx', sheet_name='количество акций', index_col=0)

In [561]:
# Функция берет столбец с ценами и возвращает столбец с изменениями цен без учета неторгуемых дней
def price_to_changes(price_path, log):
    price_path_f = price_path.copy()
    price_path_f = price_path_f[price_path_f != 0]
    if log == False:
        price_changes = price_path_f / price_path_f.shift(1) - 1
    else:
        price_changes = (price_path_f / price_path_f.shift(1)).apply(lambda x: math.log(x))
    price_changes = price_changes.dropna()
    price_changes = price_changes[price_changes != 0]
    
    return(price_changes)

In [562]:
# Функция берет дивидендную дату и, если дивиденд есть и ГЭП попадает в динамику цен, то находит примерную дату, когда был дивидендный ГЭП
def div_date_to_gap_date(div, div_date, price_changes):
    if not(np.isnan(div)):
        if (div_date >= price_changes.index[2]) and (div_date <= price_changes.index[-1]):
            div_date = price_changes[price_changes.index <= div_date].index[-1]
            possible_gap = price_changes[price_changes.index <= div_date][-3:]
            div_gap_date = possible_gap[possible_gap == possible_gap.min()].index[0]
        else:
            div_gap_date = 'За рамками динамики'
    else:
        div_gap_date = 'Нет дивиденда'
        
    return(div_gap_date)

In [563]:
# Функция добавляет в словарь параметры Моментума с указанным окном
# 'Недостаточная история торгов' - если не хватает истории котировок как минимум на половину окна моментума

def momentum_calc(selection, price, window):
    selection_f = selection.copy()
    window = 365 // 12 * window
    for date in selection_f.keys():
        param_calc = []

        for ticker in selection_f[date]['Актив']:
            dynamics = price[ticker]
            dynamics = dynamics[dynamics.index < date]
            dynamics = dynamics[dynamics.index > date - timedelta(window)]
            dynamics = dynamics[dynamics != 0]
            dif = (date - dynamics.index[0]).days

            if dif < window // 2:
                param_calc.append('Недостаточная история торгов')
            else:
                change = dynamics[-1] / dynamics[0] - 1
                param_calc.append(change)

        selection_f[date]['Momentum_' + str(int(12 / (365 // window))) + "M"] = param_calc
    
    return(selection_f)

In [564]:
# Функция добавляет в словарь параметры Volatility с указанным окном
# 'Недостаточная история торгов' - если не хватает истории котировок как минимум на половину окна Volatility

def volatility_calc(selection, price, window):
    selection_f = selection.copy()
    window = 365 // 12 * window
    for date in selection_f.keys():
        param_calc = []

        for ticker in selection_f[date]['Актив']:
            dynamics = price[ticker]
            dynamics = dynamics[dynamics.index < date]
            dynamics = dynamics[dynamics.index > date - timedelta(window)]
            dynamics = dynamics[dynamics != 0]
            dif = (date - dynamics.index[0]).days

            if dif < window // 2:
                param_calc.append('Недостаточная история торгов')
            else:
                change = price_to_changes(dynamics, log=True)
                vol = change.std() 
                param_calc.append(vol)

        selection_f[date]['Volatility_' + str(int(12 / (365 // window))) + "M"] = param_calc
    
    return(selection_f)

In [565]:
# Size - на момент балансировки: цена акции * кол-во акций
def size_calc(selection, price, Nshares):   
    selection_f = selection.copy()

    for date in selection_f.keys():
        param_calc = []
        for ticker in selection_f[date]['Актив']:
            number_shares = Nshares[Nshares.index <= date].iloc[-1][ticker]
            quote = price[price.index < date].iloc[-1][ticker]
            size = number_shares * quote
            param_calc.append(size)

        selection_f[date]['Size'] = param_calc
    
    return(selection_f)

In [566]:
# див. доходность считаем как сумма дивидендов за предыдущие 12 месяцев / цена акций на дату формирования реестра
def div_return_calc(selection, price, dividends):    
    selection_f = selection.copy()

    for date in selection_f.keys():
        date_year_ago = date - timedelta(365)
        param_calc = []
        for ticker in selection_f[date]['Актив']:
            try:
                divs_frame = dividends[(dividends['Date ' + ticker] < date) & (dividends['Date ' + ticker] > date_year_ago)]\
                                                 [['Date ' + ticker, 'Divs ' + ticker]].reset_index(drop=True)

                div_return = 0
                for d in range(len(divs_frame)):
                    div_date = divs_frame.iloc[d, 0]
                    div_value = divs_frame.iloc[d, 1]
                    stock_price = price[price.index == div_date][ticker].iloc[0]
                    if not np.isnan(stock_price) and stock_price != 0:
                        div_return += div_value / stock_price    
                param_calc.append(div_return)
            except:
                param_calc.append(0)
            
        selection_f[date]['Dividends'] = param_calc
        
    return(selection_f)

In [567]:
# Добавим в цену дивиденды. Все тикеры из таблицы с ценами должны быть в таблице с дивидендами, иначе выдаст ошибку.
tickers = price.columns.tolist()
for ticker in tickers:
    divs = dividends[['Date ' + ticker, 'Divs ' + ticker]]
    price_path = price[ticker].copy()
    price_changes = price_to_changes(price_path, log=False)
    for i in range(len(divs)):
        div_date = divs.iloc[i, 0]
        div = divs.iloc[i, 1]
        div_gap_date = div_date_to_gap_date(div, div_date, price_changes)
        if div_gap_date == 'Нет дивиденда':
            break
        if div_gap_date == 'За рамками динамики':
            continue
        else:
            price_path[price_path.index == div_gap_date] += div
    price[ticker] = price_path

In [568]:
# Частота ребалансировки (в месяцах)
freq = 3
freq_days = freq * 30
# Указываем окно для бэктеста (динамика)
# start_date должна быть на год больше, чем минимальная дата в котировках (для подсчета дивидендов за год)
# или freq месяцев больше, если не считаем фактор дивидендов
start_date = pd.to_datetime('18.12.2012')
end_date = pd.to_datetime('29.04.2022')

In [569]:
last_reb_date = start_date - timedelta(freq_days)
ac_reb_date = start_date
# min_days - столько дней должны торговаться половина компаний из индекса с момента последней ребалансировки
min_days = round(freq * 30 * 5 / 7 * 0.3)

In [570]:
# В словарь moex_rebs запишем по датам ребалансировок из нашего периода актуальные акции - Universe
universe1 = rebalancing.columns[(rebalancing.columns <= start_date).sum() - 1]
moex_rebs = {universe1 : rebalancing[universe1][[x in tickers for x in rebalancing[universe1]]].tolist()}
reb_dates = rebalancing.columns[(rebalancing.columns > start_date) & (rebalancing.columns < end_date)]
for reb_date in reb_dates:
    moex_rebs[reb_date] = rebalancing[reb_date][[x in tickers for x in rebalancing[reb_date]]].tolist()

In [571]:
# Ребалансировки индекса со списком активов, которые торговались в дату ребалансировки
real_rebs = {}
while ac_reb_date < end_date:

    moex_date = list(moex_rebs.keys())[sum([x <= ac_reb_date for x in list(moex_rebs.keys())]) - 1]
    ac_universe = moex_rebs[moex_date]
    not_trading = []

    Ntrade = 0
    Ntrade_last = 0
    for ticker in ac_universe:
        price_universe = price[ticker][price[ticker] != 0]
        price_universe = price_universe[price_universe.index <= ac_reb_date]
        price_universe = price_universe[price_universe.index >= last_reb_date]
        change_universe = (price_universe / price_universe.shift(1)).dropna()
        trade_days = (change_universe != 1).sum()
        if trade_days >= min_days:
            Ntrade += 1
        if change_universe[-1] != 1:
            Ntrade_last += 1
        else:
            not_trading.append(ticker)

    if Ntrade / len(ac_universe) >= 0.5:
        if Ntrade_last / len(ac_universe) >= 0.5:
            ac_universe = [x for x in ac_universe if x not in not_trading]
            real_rebs[ac_reb_date] = ac_universe
            last_reb_date = ac_reb_date
            ac_reb_date = ac_reb_date + timedelta(freq_days)
        else:
            ac_reb_date = ac_reb_date + timedelta(1)
    
    else:
        ac_reb_date = ac_reb_date + timedelta(freq_days)


***Расчет значений стратегий***

In [572]:
# В каждую дату ребалансировки положим датафрейм, куда будем складывать параметры стратегий через функции
selection = {}
for i in real_rebs.keys():
    selection[i] = pd.DataFrame(real_rebs[i], columns=['Актив'])

In [573]:
# Добавляем Моментум (можно несколько раз с разными окнами)
# windowM - окно в месяцах
windowM = 3
selection = momentum_calc(selection, price, windowM)

In [574]:
# Добавляем Volatility (можно несколько раз с разными окнами)
# windowV - окно в месяцах
windowV = 6
selection = volatility_calc(selection, price, windowV)

In [575]:
# Добавляем Size
selection = size_calc(selection, price, Nshares)

In [576]:
# Добавляем Dividends
selection = div_return_calc(selection, price, dividends)

In [577]:
selection

{Timestamp('2012-12-18 00:00:00'):     Актив  Momentum_3M  Volatility_6M          Size  Dividends
 0    AFKS    -0.065684       0.016326  2.398025e+11   0.012812
 1    AFLT    -0.032882       0.012270  4.834513e+10   0.039262
 2    AKRN    -0.051634       0.017067  5.516677e+10   0.069174
 3    ALRS     0.093967       0.013358  1.961806e+11   0.036996
 4    BANE    -0.039823       0.018105  3.138781e+11   0.062268
 5   BANEP     0.014087       0.012787  4.635978e+10   0.084957
 6    CHMF    -0.129111       0.017827  2.972226e+11   0.033684
 7    UPRO    -0.065120       0.018768  1.639203e+11   0.024819
 8    FEES    -0.086947       0.023714  2.680212e+11   0.000000
 9    GAZP    -0.152381       0.012907  3.307900e+12   0.051561
 10   GMKN     0.106683       0.012998  1.095680e+12   0.039023
 11   HYDR    -0.161591       0.016836  2.141853e+11   0.010541
 12   IRAO    -0.094843       0.016030  2.438716e+13   0.000000
 13   LKOH     0.026200       0.010667  1.712354e+12   0.064047
 14   

In [462]:
selection

{Timestamp('2012-12-18 00:00:00'):     Актив  Momentum_3M  Volatility_6M          Size
 0    AFKS    -0.065684       0.016326  2.398025e+11
 1    AFLT    -0.032882       0.012270  4.834513e+10
 2    AKRN    -0.051634       0.017067  5.516677e+10
 3    ALRS     0.093967       0.013358  1.961806e+11
 4    BANE    -0.039823       0.018105  3.138781e+11
 5   BANEP     0.014087       0.012787  4.635978e+10
 6    CHMF    -0.129111       0.017827  2.972226e+11
 7    UPRO    -0.065120       0.018768  1.639203e+11
 8    FEES    -0.086947       0.023714  2.680212e+11
 9    GAZP    -0.152381       0.012907  3.307900e+12
 10   GMKN     0.106683       0.012998  1.095680e+12
 11   HYDR    -0.161591       0.016836  2.141853e+11
 12   IRAO    -0.094843       0.016030  2.438716e+13
 13   LKOH     0.026200       0.010667  1.712354e+12
 14   LSRG    -0.046228       0.019123  5.888177e+10
 15   MAGN    -0.082809       0.019982  1.173305e+11
 16   MGNT     0.146667       0.019359  4.442331e+11
 17   RSTI  